# Event Log Knowledge Graph (ELKG)

See `requirements.txt` for required dependencies.  
You can install the latest `eye` reasoner from https://github.com/eyereasoner/eye/tags.

In [ ]:
import pm4py
import pandas as pd
import subprocess

from rdflib import Namespace

## CCEL (XES)

In [ ]:
# - load XES

log = pm4py.read_xes("logs/xes/sepsis.xes")

### Convert XES into ELKG

In [ ]:
from convert_log import convert_xes_rdf, RdfRepresent

convert_xes_rdf(log, "logs/xes/sepsis.ttl", Namespace("http://dutch.hospital.nl/sepsis#"), RdfRepresent.LINK_PRED)

### Test queries

In [ ]:
def run_query(query_file):
    cmd = f"eye n3/pqn.n3 --turtle logs/xes/sepsis.ttl --nope --query n3/queries/{query_file}"
    subprocess.run(cmd.split(" ")) 

In [ ]:
# - get traces based on activity occurrence, sequential relations, and arbitrary constraints

run_query("sepsis_query.n3")

## OCEL2

In [ ]:
# - load OCEL2

log = pm4py.read_ocel2("logs/ocel2/ocel2-p2p.xml")

### Convert OCEL2 log into ELKG

In [ ]:
from convert_log import convert_ocel2_rdf

# (full log)
convert_ocel2_rdf(log, "logs/ocel2/ocel2-p2p.ttl", Namespace("http://ocel2.org/p2p#"))
# # (log subset with 5000 events)
# convert_ocel2_rdf(log, "logs/ocel2/ocel2-p2p-5000.ttl", Namespace("http://ocel2.org/p2p#"), 5000)
# # (log subset with 10000 events)
# convert_ocel2_rdf(log, "logs/ocel2/ocel2-p2p-10000.ttl", Namespace("http://ocel2.org/p2p#"), 10000)

### Flatten based on perspective

In [ ]:

# - extract traces from ELKG using perspective (defined in traces_query.n3)

cmd = "eye n3/sort.n3 n3/traces/traces_collect.n3 --turtle logs/ocel2/ocel2-p2p.ttl --query n3/traces/traces_query.n3 --nope --skolem-genid trace"

with open("logs/ocel2/ocel2-p2p-traces.ttl", "w") as out_file: # file to store traces
    subprocess.run(cmd.split(" "), stdout=out_file) 

### Test queries

In [ ]:
def run_query(query_file):
    cmd = f"eye n3/pqn.n3 --turtle logs/ocel2/ocel2-p2p-expanded.ttl --nope --query n3/queries/{query_file}"
    subprocess.run(cmd.split(" ")) 

In [ ]:
# - get all events and their activities in a trace
# (n3/queries/trace_activities_query.n3)

# use to check traces in query output.
# e.g., to lookup trace t_1, update trace ID in n3/queries/trace_activities_query.n3
# and run code below

run_query("trace_activities_query.n3")

In [ ]:
# - maverick buying

run_query("p2p_mav_buying1.n3")

In [ ]:
# - maverick buying (2)

# (nothing found!)
run_query("p2p_mav_buying2.n3")

In [ ]:
# - duplicate payments

run_query("p2p_duplic_paym.n3")

In [ ]:
# - lengthy approval process

run_query("p2p_long_po_approv.n3")